# Getting started with LLaMA C++

## 1. Quick Start

Open a terminal and run the following command to check that the `llama-cli` binary is available.

```bash
which llama-cli
```

### Input prompt (One-and-done)

#### Manually downloading a model from a URL

In [1]:
%%bash

MODEL_URL=https://huggingface.co/ggml-org/gemma-1.1-7b-it-Q4_K_M-GGUF/resolve/main/gemma-1.1-7b-it.Q4_K_M.gguf
curl --location --output ../models/gemma-1.1-7b-it.Q4_K_M.gguf $MODEL_URL


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1134  100  1134    0     0   3127      0 --:--:-- --:--:-- --:--:--  3132
100 5082M  100 5082M    0     0  10.1M      0  0:08:21  0:08:21 --:--:-- 11.1M


After downloading the model file we can just pass the path to the model file as a command line argument.

-   `-m FNAME, --model FNAME`: Specify the path to the LLaMA model file.

```bash
MODEL=./models/gemma-1.1-7b-it.Q4_K_M.gguf
llama-cli --model $MODEL --prompt "Once upon a time"
```

#### Downloading a model directly from a URL

The command below makes use of the following options.

-   `-mu MODEL_URL --model-url MODEL_URL`: Specify a remote http url to download the file.

```bash
MODEL_URL=https://huggingface.co/ggml-org/gemma-1.1-7b-it-Q4_K_M-GGUF/resolve/main/gemma-1.1-7b-it.Q4_K_M.gguf
llama-cli --model-url "$MODEL_URL" --prompt "Once upon a time"
```

### Conversation mode (Allow for continuous interaction with the model)

The command below makes use of the following options.

- `-cnv,  --conversation`: run in conversation mode:
  - does not print special tokens and suffix/prefix
  - interactive mode is also enabled.
- `--chat-template JINJA_TEMPLATE`: Set custom jinja chat template (default: template taken from model's metadata) if suffix/prefix are specified, see the [LLaMA C++ documentation](https://github.com/ggerganov/llama.cpp/wiki/Templates-supported-by-llama_chat_apply_template) for the current list of accepted chat templates.

```bash
CHAT_TEMPLATE=gemma
llama-cli --model $MODEL --conversation --chat-template $CHAT_TEMPLATE
```

## 2. Input Prompts

The `llama-cli` program provides several ways to interact with the LLaMA models using input prompts:

-   `--prompt PROMPT`: Provide a prompt directly as a command-line option.
-   `--file FNAME`: Provide a file containing a prompt or multiple prompts.
-   `--interactive-first`: Run the program in interactive mode and wait for input right away. (More on this below.)

### `--prompt` example

```bash
llama-cli --model "$MODEL" --prompt "What is the meaning of life?"
```

### `--file` example

In [5]:
language = "English"
tone_of_voice = "Informative"
topic = "Computer Science"
writing_style = "Conversational"

prompt_template = f"""Please ignore all previous instructions. Please respond \
only in the {language} language. You are a Twitter influencer with a large \
following. You have a {tone_of_voice} tone of voice. You have a \
{writing_style} writing style. Do not self reference. Do not explain what you \
are doing. Please create a thread about {topic}. Add emojis to the thread \
when appropriate. The character count for each thread should be between 270 \
to 280 characters. Your content should be casual, informative, and an \
engaging Twitter thread. Please use simple and understandable words. Please \
include statistics, personal experience, and fun facts in the thread. Please \
add relevant hashtags to the post and encourage the readers join the \
conversation.
"""


In [6]:
print(prompt_template)

Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.



In [7]:
with open("../prompts/engaging-twitter-thread.txt", 'w') as f:
    f.write(prompt_template)


```bash
llama-cli --model "$MODEL" --file ./prompts/engaging-twitter-thread.txt
```

### `--interactive-first` example

```bash
llama-cli --model "$MODEL" --interactive-first
```

## 3. Interaction

The `llama-cli` program offers a seamless way to interact with LLaMA models, allowing users to engage in real-time conversations or provide instructions for specific tasks. The interactive mode can be triggered using various options, including `--interactive` and `--interactive-first`.

In interactive mode, users can participate in text generation by injecting their input during the process. Users can press `Ctrl+C` at any time to interject and type their input, followed by pressing `Return` to submit it to the LLaMA model. To submit additional lines without finalizing input, users can end the current line with a backslash (`\`) and continue typing.

### Interaction Options

-   `-i, --interactive`: Run the program in interactive mode, allowing users to engage in real-time conversations or provide specific instructions to the model.
-   `--interactive-first`: Run the program in interactive mode and immediately wait for user input before starting the text generation.
-   `-cnv,  --conversation`:  Run the program in conversation mode (does not print special tokens and suffix/prefix, use default chat template) (default: false)
-   `--color`: Enable colorized output to differentiate visually distinguishing between prompts, user input, and generated text.

By understanding and utilizing these interaction options, you can create engaging and dynamic experiences with your models, tailoring the text generation process to your specific needs.

### Example

Type the following command in the terminal.

```bash
llama-cli --model "$MODEL" --conversation --color
```

### Reverse Prompts

Reverse prompts are a powerful way to create a chat-like experience with your model by pausing the text generation when specific text strings are encountered using the `--reverse-prompt` option.

-   `-r PROMPT, --reverse-prompt PROMPT`: Specify one or multiple reverse prompts to pause text generation and switch to interactive mode. For example, `-r "User:"` can be used to jump back into the conversation whenever it's the user's turn to speak. This helps create a more interactive and conversational experience.


In [ ]:
#TODO: usage example!

### In-Prefix

The `--in-prefix` flag is used to add a prefix to your input, primarily, this is used to insert a space after the reverse prompt. Here's an example of how to use the `--in-prefix` flag in conjunction with the `--reverse-prompt` flag:

```sh
llama-cli -r "User:" --in-prefix " "
```

In [ ]:
#TODO: usage example!

### In-Suffix

The `--in-suffix` flag is used to add a suffix after your input. This is useful for adding an "Assistant:" prompt after the user's input. It's added after the new-line character (`\n`) that's automatically added to the end of the user's input. Here's an example of how to use the `--in-suffix` flag in conjunction with the `--reverse-prompt` flag:

```sh
./llama-cli -r "User:" --in-prefix " " --in-suffix "Assistant:"
```
When --in-prefix or --in-suffix options are enabled the chat template ( --chat-template ) is disabled

In [ ]:
#TODO: usage example!

### Chat templates

 `--chat-template JINJA_TEMPLATE`: This option sets a custom jinja chat template. It accepts a string, not a file name.  Default: template taken from model's metadata. Llama.cpp only supports [some pre-defined templates](https://github.com/ggerganov/llama.cpp/wiki/Templates-supported-by-llama_chat_apply_template). These include the following

- `llama2`
- `llama3`
- `gemma`
- `monarch`
- `chatml`
- `orion`
- `vicuna`
- `vicuna-orca`
- `deepseek`
- `command-r`
- `zephyr`

When `--in-prefix` or `--in-suffix` options are enabled the chat template ( `--chat-template` ) is disabled.


In [ ]:
#TODO: usage example!

## 4. Context Management

During text generation, models have a limited context size, which means they can only consider a certain number of tokens from the input and generated text. When the context fills up, the model resets internally,  otentially losing some information from the beginning of the conversation or instructions. Context management options help maintain continuity and coherence in these situations.

### Context Size

- `-c N, --ctx-size N`: Set the size of the prompt context (default: 0, 0 = loaded from model). The LLaMA models were built with a context of 2048-8192, which will yield the best results on longer input/inference.

In [8]:
#TODO: usage example

### Extended Context Size

Some fine-tuned models have extended the context length by scaling RoPE. For example, if the original pre-trained model has a context length (max sequence length) of 4096 (4k) and the fine-tuned model has 32k. That is a scaling factor of 8, and should work by setting the above `--ctx-size` to 32768 (32k) and `--rope-scale` to 8.

-   `--rope-scale N`: Where N is the linear scaling factor used by the fine-tuned model.

In [ ]:
#TODO: usage example!

### Keep Prompt

The `--keep` option allows users to retain the original prompt when the model runs out of context, ensuring a connection to the initial instruction or conversation topic is maintained.

-   `--keep N`: Specify the number of tokens from the initial prompt to retain when the model resets its internal context. By default, this value is set to 0 (meaning no tokens are kept). Use `-1` to retain all tokens from the initial prompt.

By utilizing context management options like `--ctx-size` and `--keep`, you can maintain a more coherent and consistent interaction with the LLaMA models, ensuring that the generated text remains relevant to the original prompt or conversation.

In [ ]:
#TODO: usage example!

## 5. Generation Flags

The following options allow you to control the text generation process and fine-tune the diversity, creativity, and quality of the generated text according to your needs. By adjusting these options and experimenting with different combinations of values, you can find the best settings for your specific use case.

In [18]:
MODEL = "../models/gemma-1.1-7b-it.Q4_K_M.gguf"

### Random Number Generator (RNG) Seed

-   `-s SEED, --seed SEED`: Set the random number generator (RNG) seed (default: -1, -1 = random seed).

The RNG seed is used to initialize the random number generator that influences the text generation process. By setting a specific `--seed` value, you can obtain consistent and reproducible results across multiple runs with the same input and settings. This can be helpful for testing, debugging, or comparing the effects of different options on the generated text to see when they diverge. If the seed is set to a value less than 0, a random seed will be used, which will result in different outputs on each run. The default value is -1 which will choose a random value for `--seed`.

#### Random `--seed` example

In [61]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --seed -1 \
    --color \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

## Thread: The Magic of Computer Science ✨💻

Ever wondered how apps translate your language to code? Or how websites load in a blink? That's the power of **Computer Science**! 💻📚

It's the building block of the digital age, shaping everything from your smartphone to t

llama_perf_sampler_print:    sampling time =      20.24 ms /   366 runs   (    0.06 ms per token, 18085.68 tokens per second)
llama_perf_context_print:        load time =    2310.27 ms
llama_perf_context_print: prompt eval time =    1131.81 ms /   141 tokens (    8.03 ms per token,   124.58 tokens per second)
llama_perf_context_print:        eval time =   14383.53 ms /   224 runs   (   64.21 ms per token,    15.57 tokens per second)
llama_perf_context_print:       total time =   15548.33 ms /   365 tokens
ggml_metal_free: deallocating


### Fixed `--seed` example

In [64]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --seed 42 \
    --color \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet 1/5**

Hey there, code enthusiasts! 💻✨ Ever wondered how apps keep running smoothly or how complex websites load in seconds? The magic behind that is **Computer Science**! 💪 It's the science of designing, building, and maintaining the software that powers our 

llama_perf_sampler_print:    sampling time =      33.70 ms /   508 runs   (    0.07 ms per token, 15073.29 tokens per second)
llama_perf_context_print:        load time =    2018.41 ms
llama_perf_context_print: prompt eval time =    1131.30 ms /   141 tokens (    8.02 ms per token,   124.64 tokens per second)
llama_perf_context_print:        eval time =   24026.85 ms /   366 runs   (   65.65 ms per token,    15.23 tokens per second)
llama_perf_context_print:       total time =   25211.86 ms /   507 tokens
ggml_metal_free: deallocating


### Number of Tokens to Predict

The `-n N, --predict N` (default: -1) controls the number of tokens the model generates in response to the input prompt. By adjusting this value, you can influence the length of the generated text. A higher value will result in longer text, while a lower value will produce shorter text.

Even though all models have a finite context window, a value of -1 will enable *infinite* text generation. How? When the context window is full, some of the earlier tokens (half of the tokens after `--keep`) will be discarded. The context must then be re-evaluated before generation can resume. On large models and/or large context windows, this can result in a significant pause in output. If the output delay is undesirable, a value of -2 will stop generation immediately when the context is filled.

It is important to note that the generated text may be shorter than the specified number of tokens if an End-of-Sequence (EOS) token or a reverse prompt is encountered. In interactive mode, text generation will pause and control will be returned to the user. In non-interactive mode, the program will end. In both cases, the text generation may stop before reaching the specified `--predict` value. If you want the model to keep going without ever producing End-of-Sequence on its own, you can use the `--ignore-eos` parameter.

#### Basic `--predict` example

```bash
llama-cli --model "$MODEL" --predict 10 --prompt "What is the meaning of life?"
```

#### "until context filled" text generation example

```bash
llama-cli --model "$MODEL" --ctx-size 10 --predict -2 --prompt "What is the meaning of life?"
```

#### "Infinite" text generation example

```bash
llama-cli --model "$MODEL" --ctx-size 10 --predict -1 --prompt "What is the meaning of life?"
```

### Temperature

-   `--temp N`: Adjust the randomness of the generated text (default: 0.8).

Temperature is a hyperparameter that controls the randomness of the generated text. It affects the probability distribution of the model's output tokens. A higher temperature (e.g., 1.5) makes the output more random and creative, while a lower temperature (e.g., 0.5) makes the output more focused, deterministic, and conservative. The default value is 0.8, which provides a balance between randomness and determinism. At the extreme, a temperature of 0 will always pick the most likely next token, leading to identical outputs in each run.

#### Default `--temp` example

In [65]:
%%bash -s "$MODEL"

llama-cli --model "$1" --temp 0.8 --color --file ../prompts/engaging-twitter-thread.txt

build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet 1/5**

Hey there, code enthusiasts! 💻✨ Ever wondered how apps remember your preferences or how websites suggest products you might love? That's the magic of Computer Science! 💫 It's the science behind the algorithms that power our digital world. #CSisEverywher

llama_perf_sampler_print:    sampling time =      29.47 ms /   466 runs   (    0.06 ms per token, 15814.84 tokens per second)
llama_perf_context_print:        load time =    1860.84 ms
llama_perf_context_print: prompt eval time =    1135.05 ms /   141 tokens (    8.05 ms per token,   124.22 tokens per second)
llama_perf_context_print:        eval time =   21173.36 ms /   324 runs   (   65.35 ms per token,    15.30 tokens per second)
llama_perf_context_print:       total time =   22356.97 ms /   465 tokens
ggml_metal_free: deallocating


#### Low `--temp` example

In [32]:
%%bash -s "$MODEL"

llama-cli --model "$1" --temp 0.4 --color --file ../prompts/engaging-twitter-thread.txt

build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet 1:**

Hey there, code enthusiasts! 💻✨ Ever wondered how apps remember your preferences or how websites suggest products you might love? That's the magic of **Computer Science** in action! 💪 It's the science of creating intelligent systems that can learn and ad

llama_perf_sampler_print:    sampling time =      26.20 ms /   421 runs   (    0.06 ms per token, 16071.77 tokens per second)
llama_perf_context_print:        load time =    2031.56 ms
llama_perf_context_print: prompt eval time =    1131.14 ms /   141 tokens (    8.02 ms per token,   124.65 tokens per second)
llama_perf_context_print:        eval time =   17992.20 ms /   279 runs   (   64.49 ms per token,    15.51 tokens per second)
llama_perf_context_print:       total time =   19165.60 ms /   420 tokens
ggml_metal_free: deallocating


#### High `--temp` example

In [33]:
%%bash -s "$MODEL"

llama-cli --model "$1" --temp 1.4 --color --file ../prompts/engaging-twitter-thread.txt

build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation. 

**Tweet 1/5**

Hey there, code lovers! 💻 Ever wondered what makes the apps you use tick? 🤔 That's the magic of Computer Science! 💪 It's the art of crafting algorithms and building software that shapes our digital world 🌎 From Instagram filters to online banking, CS i

llama_perf_sampler_print:    sampling time =      32.85 ms /   507 runs   (    0.06 ms per token, 15434.26 tokens per second)
llama_perf_context_print:        load time =    1947.13 ms
llama_perf_context_print: prompt eval time =    1137.36 ms /   141 tokens (    8.07 ms per token,   123.97 tokens per second)
llama_perf_context_print:        eval time =   23653.09 ms /   365 runs   (   64.80 ms per token,    15.43 tokens per second)
llama_perf_context_print:       total time =   24841.23 ms /   506 tokens
ggml_metal_free: deallocating


### Repeat Penalty

The `--repeat-penalty` option helps prevent the model from generating repetitive or monotonous text. A higher value (e.g., 1.5) will penalize repetitions more strongly, while a lower value (e.g., 0.9) will be more lenient. The default value is 1 (which means no penalty).

The `--repeat-last-n` option controls the number of tokens in the history to consider for penalizing repetition. A larger value will look further back in the generated text to prevent repetitions, while a smaller value will only consider recent tokens. A value of 0 disables the penalty, and a value of -1 sets the number of tokens considered equal to the context size, `--ctx-size`. The default value is 64. 


In [35]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --repeat-penalty 1.5 \
    --repeat-last-n 128 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet #1:**
Hey there! 👋 Ever wondered how your phone can translate languages or suggest songs based on mood? That's magic powered by **Computer science (CS)** 💪 It deals with building & designing digital systems that shape our daily lives ✨ From apps you love  to c

llama_perf_sampler_print:    sampling time =      69.41 ms /   261 runs   (    0.27 ms per token,  3760.05 tokens per second)
llama_perf_context_print:        load time =    1765.60 ms
llama_perf_context_print: prompt eval time =    1132.28 ms /   141 tokens (    8.03 ms per token,   124.53 tokens per second)
llama_perf_context_print:        eval time =    7643.48 ms /   119 runs   (   64.23 ms per token,    15.57 tokens per second)
llama_perf_context_print:       total time =    8852.79 ms /   260 tokens
ggml_metal_free: deallocating


### Top-K Sampling

Top-k sampling is a text generation method that selects the next token only from the `--top-k` most likely tokens predicted by the model. It helps reduce the risk of generating low-probability or nonsensical tokens, but it may also limit the diversity of the output. A higher value for top-k (e.g., 100) will consider more tokens and lead to more diverse text, while a lower value (e.g., 10) will focus on the most probable tokens and generate more conservative text. The default value is 40.


In [36]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --top-k 32 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation. 

**Tweet 1/5**

Hey there, code enthusiasts! 💻 Ever wondered what makes those apps you love tick? That's the magic of **Computer Science** 🧙‍♀️! It's the science of designing, building, and maintaining the software and hardware that shapes our digital world 🌍. #CSisFu

llama_perf_sampler_print:    sampling time =      33.68 ms /   470 runs   (    0.07 ms per token, 13953.63 tokens per second)
llama_perf_context_print:        load time =    1751.28 ms
llama_perf_context_print: prompt eval time =    1126.72 ms /   141 tokens (    7.99 ms per token,   125.14 tokens per second)
llama_perf_context_print:        eval time =   21196.33 ms /   328 runs   (   64.62 ms per token,    15.47 tokens per second)
llama_perf_context_print:       total time =   22375.40 ms /   469 tokens
ggml_metal_free: deallocating


In [37]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --top-k 10 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

## Thread: The Fascinating World of Computer Science 💻📚

Ever wondered how apps remember your preferences? Or how websites load in seconds? That's the magic of Computer Science! 💪

It's the building block of everything digital, from the apps we use daily to the robots

llama_perf_sampler_print:    sampling time =      20.75 ms /   398 runs   (    0.05 ms per token, 19177.95 tokens per second)
llama_perf_context_print:        load time =    1729.98 ms
llama_perf_context_print: prompt eval time =    1145.92 ms /   141 tokens (    8.13 ms per token,   123.04 tokens per second)
llama_perf_context_print:        eval time =   16575.67 ms /   256 runs   (   64.75 ms per token,    15.44 tokens per second)
llama_perf_context_print:       total time =   17756.02 ms /   397 tokens
ggml_metal_free: deallocating


In [38]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --top-k 100 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet 1/5**

Hey there, code enthusiasts! 💻✨ Did you know there are over 23 million computer scientists globally? 🤯 That's a whole lotta minds solving problems and shaping the future! #CSCommunity #TechLife

**Tweet 2/5**

The beauty of computer science is its versa

llama_perf_sampler_print:    sampling time =      30.54 ms /   422 runs   (    0.07 ms per token, 13818.40 tokens per second)
llama_perf_context_print:        load time =    1575.63 ms
llama_perf_context_print: prompt eval time =    1130.34 ms /   141 tokens (    8.02 ms per token,   124.74 tokens per second)
llama_perf_context_print:        eval time =   18060.41 ms /   280 runs   (   64.50 ms per token,    15.50 tokens per second)
llama_perf_context_print:       total time =   19236.56 ms /   421 tokens
ggml_metal_free: deallocating


### Top-P Sampling

-   `--top-p N`: Limit the next token selection to a subset of tokens with a cumulative probability above a threshold P (default: 0.9).

Top-p sampling, also known as nucleus sampling, is another text generation method that selects the next token from a subset of tokens that together have a cumulative probability of at least p. This method provides a balance between diversity and quality by considering both the probabilities of tokens and the number of tokens to sample from. A higher value for top-p (e.g., 0.95) will lead to more diverse text, while a lower value (e.g., 0.5) will generate more focused and conservative text. The default value is 0.9.

Example usage: `--top-p 0.95`

#### Default `--top-p` example

In [39]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --top-p 0.9 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

## Thread: The Magical World of Computer Science 💻✨

Ever wondered how apps respond instantly to your touch? Or how websites load in a flash? That's the magic of **Computer Science** 💻!

It's the science of designing, building, and maintaining the software and systems

llama_perf_sampler_print:    sampling time =      19.25 ms /   353 runs   (    0.05 ms per token, 18332.90 tokens per second)
llama_perf_context_print:        load time =    1716.48 ms
llama_perf_context_print: prompt eval time =    1131.20 ms /   141 tokens (    8.02 ms per token,   124.65 tokens per second)
llama_perf_context_print:        eval time =   13688.10 ms /   211 runs   (   64.87 ms per token,    15.41 tokens per second)
llama_perf_context_print:       total time =   14851.37 ms /   352 tokens
ggml_metal_free: deallocating


#### Low `--top-p` example

In [40]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --top-p 0.5 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet 1/5**

Hey there, code enthusiasts! 💻✨ Ever wondered how apps remember your preferences or how websites suggest products you might love? That's the magic of Computer Science! 📚 It's the foundation of everything that makes technology personalized and interactiv

llama_perf_sampler_print:    sampling time =      28.80 ms /   459 runs   (    0.06 ms per token, 15939.71 tokens per second)
llama_perf_context_print:        load time =    1919.87 ms
llama_perf_context_print: prompt eval time =    1131.24 ms /   141 tokens (    8.02 ms per token,   124.64 tokens per second)
llama_perf_context_print:        eval time =   20418.50 ms /   317 runs   (   64.41 ms per token,    15.53 tokens per second)
llama_perf_context_print:       total time =   21595.23 ms /   458 tokens
ggml_metal_free: deallocating


#### High `--top-p` example

In [41]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --top-p 0.95 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet 1:**

Hey there, code enthusiasts! 💻 Ever wondered what makes apps run so smoothly or how websites load in a flash? Enter: Computer Science! 💪 It's the magic behind the tech we take for granted. 🧙‍♀️ #CSUnbound #TechMagic

**Tweet 2:**

Did you know? 🤯 Over 23

llama_perf_sampler_print:    sampling time =      37.79 ms /   494 runs   (    0.08 ms per token, 13072.59 tokens per second)
llama_perf_context_print:        load time =    1665.69 ms
llama_perf_context_print: prompt eval time =    1130.17 ms /   141 tokens (    8.02 ms per token,   124.76 tokens per second)
llama_perf_context_print:        eval time =   23281.70 ms /   352 runs   (   66.14 ms per token,    15.12 tokens per second)
llama_perf_context_print:       total time =   24472.78 ms /   493 tokens
ggml_metal_free: deallocating


### Min-P Sampling

The `--min-p` sampling method sets a minimum base probability threshold for token selection and aims to ensure a balance of quality and variety in the generated text. The `--min-p` method was designed as an alternative to `--top-p`. The parameter *p* represents the minimum probability for a token to be considered, relative to the probability of the most likely token. For example, with *p*=0.05 and the most likely token having a probability of 0.9, logits with a value less than 0.045 are filtered out. The default value is 0.1.


#### Default `--min-p` example

In [42]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --top-p 0.9 \
    --min-p 0.1 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

## Thread: The Magic of Computer Science 💻✨

Imagine a world where problems solve themselves, machines learn like humans, and creativity meets technology. That's the power of **Computer Science** 💪! 

Did you know? ➡️ 23% of the fastest-growing jobs in the US require 

llama_perf_sampler_print:    sampling time =      28.37 ms /   435 runs   (    0.07 ms per token, 15330.94 tokens per second)
llama_perf_context_print:        load time =    2531.79 ms
llama_perf_context_print: prompt eval time =    1126.58 ms /   141 tokens (    7.99 ms per token,   125.16 tokens per second)
llama_perf_context_print:        eval time =   19150.07 ms /   293 runs   (   65.36 ms per token,    15.30 tokens per second)
llama_perf_context_print:       total time =   20322.09 ms /   434 tokens
ggml_metal_free: deallocating


#### Low `--min-p` example

In [43]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --top-p 0.9 \
    --min-p 0.05 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

## Thread: The Magic of Computer Science 💻📚✨

Ever wondered how apps remember your preferences or how websites know exactly what you're searching for? That's the magic of **Computer Science (CS)**! 🤯

CS is about building the technology that shapes our world. It's abo

llama_perf_sampler_print:    sampling time =      18.46 ms /   336 runs   (    0.05 ms per token, 18201.52 tokens per second)
llama_perf_context_print:        load time =    1797.62 ms
llama_perf_context_print: prompt eval time =    1132.28 ms /   141 tokens (    8.03 ms per token,   124.53 tokens per second)
llama_perf_context_print:        eval time =   12484.22 ms /   194 runs   (   64.35 ms per token,    15.54 tokens per second)
llama_perf_context_print:       total time =   13646.65 ms /   335 tokens
ggml_metal_free: deallocating


#### High `--min-p` example

In [44]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --top-p 0.9 \
    --min-p 0.2 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet 1/5**

Hey there, code enthusiasts! 💻 Ever wondered how apps seamlessly translate languages or suggest music you'll love? That's the magic of Computer Science! 💪 It's about building intelligent systems that understand and interact with humans like magic 💫 #CSI

llama_perf_sampler_print:    sampling time =      27.15 ms /   435 runs   (    0.06 ms per token, 16021.51 tokens per second)
llama_perf_context_print:        load time =    1831.47 ms
llama_perf_context_print: prompt eval time =    1136.34 ms /   141 tokens (    8.06 ms per token,   124.08 tokens per second)
llama_perf_context_print:        eval time =   18995.89 ms /   293 runs   (   64.83 ms per token,    15.42 tokens per second)
llama_perf_context_print:       total time =   20174.87 ms /   434 tokens
ggml_metal_free: deallocating


In [50]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --keep 1.0 \
    --tfs 1.0 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet 1/5**

Hey there, code enthusiasts! 💻 Ever wondered how apps seamlessly process data or how websites load in an instant? That's the magic of Computer Science! 💪 It's the building block of our digital world, shaping everything from entertainment to healthcare. 

llama_perf_sampler_print:    sampling time =      31.76 ms /   481 runs   (    0.07 ms per token, 15143.88 tokens per second)
llama_perf_context_print:        load time =    3547.38 ms
llama_perf_context_print: prompt eval time =    1142.52 ms /   141 tokens (    8.10 ms per token,   123.41 tokens per second)
llama_perf_context_print:        eval time =   21981.09 ms /   339 runs   (   64.84 ms per token,    15.42 tokens per second)
llama_perf_context_print:       total time =   23174.22 ms /   480 tokens
ggml_metal_free: deallocating


### Locally Typical Sampling

Locally typical sampling, `--typical` promotes the generation of contextually coherent and diverse text by sampling tokens that are typical or expected based on the surrounding context. By setting the parameter $p$ between 0 and 1, you can control the balance between producing text that is locally coherent and diverse.

#### Default `--typical` example

The default value of 1 disables locally typical sampling.


In [ ]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --typical 1.0 \
    --file ../prompts/engaging-twitter-thread.txt


#### Typical `--typical` example

A value closer to 1 will promote more contextually coherent tokens.

In [58]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --typical 0.9 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

## Thread: The Superpowers of Computer Science 💪💻

Ever wondered how apps remember your preferences, websites load in a flash, or robots can perform complex tasks? That's the magic of **Computer Science**! 💻✨

It's the science of designing, building, and operating dig

llama_perf_sampler_print:    sampling time =      20.21 ms /   365 runs   (    0.06 ms per token, 18055.90 tokens per second)
llama_perf_context_print:        load time =    3451.53 ms
llama_perf_context_print: prompt eval time =    1128.57 ms /   141 tokens (    8.00 ms per token,   124.94 tokens per second)
llama_perf_context_print:        eval time =   14321.78 ms /   223 runs   (   64.22 ms per token,    15.57 tokens per second)
llama_perf_context_print:       total time =   15483.71 ms /   364 tokens
ggml_metal_free: deallocating


#### Low `--typical` example

A `--typical` value closer to 0 will promote more diverse tokens. 

In [59]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --typical 0.25 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

## Thread: 1/5

Hey there, code enthusiasts! 💻✨ Ever wondered how apps remember your preferences or how websites suggest products you might love? That's the magic of computer science! 💪 It's the science of building intelligent machines that can learn, adapt, and solve

llama_perf_sampler_print:    sampling time =      31.49 ms /   478 runs   (    0.07 ms per token, 15179.42 tokens per second)
llama_perf_context_print:        load time =    2354.31 ms
llama_perf_context_print: prompt eval time =    1129.27 ms /   141 tokens (    8.01 ms per token,   124.86 tokens per second)
llama_perf_context_print:        eval time =   21873.39 ms /   336 runs   (   65.10 ms per token,    15.36 tokens per second)
llama_perf_context_print:       total time =   23053.39 ms /   477 tokens
ggml_metal_free: deallocating


### Mirostat Sampling

Mirostat is an algorithm that actively maintains the quality of generated text within a desired range during text generation. It aims to strike a balance between coherence and diversity, avoiding low-quality output caused by excessive repetition (boredom traps) or incoherence (confusion traps). To enable Mirostat sampling set `--mirostat` to 1 = Mirostat 1.0 or 2 = Mirostat 2.0. By default Mirostat sampling is disabled, `--mirostat 0`.

The `--mirostat-lr` option sets the Mirostat learning rate (eta). The learning rate influences how quickly the algorithm responds to feedback from the generated text. A lower learning rate will result in slower adjustments, while a higher learning rate will make the algorithm more responsive. The default value is `0.1`.

The `--mirostat-ent` option sets the Mirostat target entropy (tau), which represents the desired perplexity value for the generated text. Adjusting the target entropy allows you to control the balance between coherence and diversity in the generated text. A lower value will result in more focused and coherent text, while a higher value will lead to more diverse and potentially less coherent text. The default value is `5.0`.

### Example

In [60]:
%%bash -s "$MODEL"

llama-cli \
    --model "$1" \
    --color \
    --mirostat 2 \
    --mirostat-lr 0.05 \
    --mirostat-ent 3.0 \
    --file ../prompts/engaging-twitter-thread.txt


build: 3865 (00b7317e) with Apple clang version 16.0.0 (clang-1600.0.26.3) for arm64-apple-darwin24.0.0
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 24 key-value pairs and 254 tensors from ../models/gemma-1.1-7b-it.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-1.1-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   4:                          gemma.block_count u32              = 28
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32   

 Please ignore all previous instructions. Please respond only in the English language. You are a Twitter influencer with a large following. You have a Informative tone of voice. You have a Conversational writing style. Do not self reference. Do not explain what you are doing. Please create a thread about Computer Science. Add emojis to the thread when appropriate. The character count for each thread should be between 270 to 280 characters. Your content should be casual, informative, and an engaging Twitter thread. Please use simple and understandable words. Please include statistics, personal experience, and fun facts in the thread. Please add relevant hashtags to the post and encourage the readers join the conversation.

**Tweet 1/5:**

Hey there, code enthusiasts! 💻✨ Ever wondered how apps seamlessly translate your language into different languages? That's the magic of Natural Language Processing (NLP)! 💪 NLP algorithms analyze language patterns to understand meaning & translate seam

llama_perf_sampler_print:    sampling time =    4944.35 ms /   450 runs   (   10.99 ms per token,    91.01 tokens per second)
llama_perf_context_print:        load time =    2032.12 ms
llama_perf_context_print: prompt eval time =    1138.22 ms /   141 tokens (    8.07 ms per token,   123.88 tokens per second)
llama_perf_context_print:        eval time =   19937.24 ms /   308 runs   (   64.73 ms per token,    15.45 tokens per second)
llama_perf_context_print:       total time =   26037.06 ms /   449 tokens
ggml_metal_free: deallocating


## 6. Performance Tuning and Memory Options

These options help improve the performance and memory usage of the LLaMA models. By adjusting these settings, you can fine-tune the model's behavior to better suit your system's capabilities and achieve optimal performance for your specific use case.

### Number of Threads

-   `-t N, --threads N`: Set the number of threads to use during generation. For optimal performance, it is recommended to set this value to the number of physical CPU cores your system has (as opposed to the logical number of cores). Using the correct number of threads can greatly improve performance.
-   `-tb N, --threads-batch N`: Set the number of threads to use during batch and prompt processing. In some systems, it is beneficial to use a higher number of threads during batch processing than during generation. If not specified, the number of threads used for batch processing will be the same as the number of threads used for generation.

### Mlock

-   `--mlock`: Lock the model in memory, preventing it from being swapped out when memory-mapped. This can improve performance but trades away some of the advantages of memory-mapping by requiring more RAM to run and potentially slowing down load times as the model loads into RAM.

### No Memory Mapping

-   `--no-mmap`: Do not memory-map the model. By default, models are mapped into memory, which allows the system to load only the necessary parts of the model as needed. However, if the model is larger than your total amount of RAM or if your system is low on available memory, using mmap might increase the risk of pageouts, negatively impacting performance. Disabling mmap results in slower load times but may reduce pageouts if you're not using `--mlock`. Note that if the model is larger than the total amount of RAM, turning off mmap would prevent the model from loading at all.

### NUMA support

-   `--numa distribute`: Pin an equal proportion of the threads to the cores on each NUMA node. This will spread the load amongst all cores on the system, utilitizing all memory channels at the expense of potentially requiring memory to travel over the slow links between nodes.
-   `--numa isolate`: Pin all threads to the NUMA node that the program starts on. This limits the number of cores and amount of memory that can be used, but guarantees all memory access remains local to the NUMA node.
-   `--numa numactl`: Pin threads to the CPUMAP that is passed to the program by starting it with the numactl utility. This is the most flexible mode, and allow arbitrary core usage patterns, for example a map that uses all the cores on one NUMA nodes, and just enough cores on a second node to saturate the inter-node memory bus.

 These flags attempt optimizations that help on some systems with non-uniform memory access. This currently consists of one of the above strategies, and disabling prefetch and readahead for mmap. The latter causes mapped pages to be faulted in on first access instead of all at once, and in combination with pinning threads to NUMA nodes, more of the pages end up on the NUMA node where they are used. Note that if the model is already in the system page cache, for example because of a previous run without this option, this will have little effect unless you drop the page cache first. This can be done by rebooting the system or on Linux by writing '3' to '/proc/sys/vm/drop_caches' as root.

### Batch Size

-   `-b N, --batch-size N`: Set the batch size for prompt processing (default: `2048`). This large batch size benefits users who have BLAS installed and enabled it during the build. If you don't have BLAS enabled ("BLAS=0"), you can use a smaller number, such as 8, to see the prompt progress as it's evaluated in some situations.

- `-ub N`, `--ubatch-size N`: physical maximum batch size. This is for pipeline parallelization. Default: `512`.

### Prompt Caching

-   `--prompt-cache FNAME`: Specify a file to cache the model state after the initial prompt. This can significantly speed up the startup time when you're using longer prompts. The file is created during the first run and is reused and updated in subsequent runs. **Note**: Restoring a cached prompt does not imply restoring the exact state of the session at the point it was saved. So even when specifying a specific seed, you are not guaranteed to get the same sequence of tokens as the original generation.

### Grammars & JSON schemas

-   `--grammar GRAMMAR`, `--grammar-file FILE`: Specify a grammar (defined inline or in a file) to constrain model output to a specific format. For example, you could force the model to output JSON or to speak only in emojis. See the [GBNF guide](../../grammars/README.md) for details on the syntax.

-   `--json-schema SCHEMA`: Specify a [JSON schema](https://json-schema.org/) to constrain model output to (e.g. `{}` for any JSON object, or `{"items": {"type": "string", "minLength": 10, "maxLength": 100}, "minItems": 10}` for a JSON array of strings with size constraints). If a schema uses external `$ref`s, you should use `--grammar "$( python examples/json_schema_to_grammar.py myschema.json )"` instead.

## 6. Additional Options

These options provide extra functionality and customization when running the LLaMA models:

-   `-h, --help`: Display a help message showing all available options and their default values. This is particularly useful for checking the latest options and default values, as they can change frequently, and the information in this document may become outdated.
-   `--verbose-prompt`: Print the prompt before generating text.
-   `-mg i, --main-gpu i`: When using multiple GPUs this option controls which GPU is used for small tensors for which the overhead of splitting the computation across all GPUs is not worthwhile. The GPU in question will use slightly more VRAM to store a scratch buffer for temporary results. By default GPU 0 is used.
-   `-ts SPLIT, --tensor-split SPLIT`: When using multiple GPUs this option controls how large tensors should be split across all GPUs. `SPLIT` is a comma-separated list of non-negative values that assigns the proportion of data that each GPU should get in order. For example, "3,2" will assign 60% of the data to GPU 0 and 40% to GPU 1. By default the data is split in proportion to VRAM but this may not be optimal for performance.
-   `--lora FNAME`: Apply a LoRA (Low-Rank Adaptation) adapter to the model (implies --no-mmap). This allows you to adapt the pretrained model to specific tasks or domains.
-   `--lora-base FNAME`: Optional model to use as a base for the layers modified by the LoRA adapter. This flag is used in conjunction with the `--lora` flag, and specifies the base model for the adaptation.
-   `-hfr URL --hf-repo URL`: The url to the Hugging Face model repository. Used in conjunction with `--hf-file` or `-hff`. The model is downloaded and stored in the file provided by `-m` or `--model`. If `-m` is not provided, the model is auto-stored in the path specified by the `LLAMA_CACHE` environment variable  or in an OS-specific local cache.